In [2]:
import tensorflow as tf
import yaml
from modeling.model import AttributeMaskrcnnModel  # Assuming this is your custom model class

# Step 1: Load parameters from a YAML file
with open('configs/yaml/spinenet143_amrcnn.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Step 2: Create the TensorFlow 2.x model instance
model = AttributeMaskrcnnModel(params)

# Step 3: Load the TF1.x checkpoint into the model
checkpoint = tf.train.Checkpoint(model=model)
checkpoint_path = 'checkpoints/fashionpedia-spinenet-143/model.ckpt'
checkpoint.restore(checkpoint_path).expect_partial()  # Restore TF1.x weights into TF2.x model

# Step 4: Save the model in SavedModel format
saved_model_dir = "saved_models_tf2"
tf.saved_model.save(model, saved_model_dir)

# Step 5: Verify the SavedModel (optional)
loaded_model = tf.saved_model.load(saved_model_dir)


C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


AttributeError: 'dict' object has no attribute 'train'

In [17]:
tf.compat.v1.disable_eager_execution()

In [25]:
from configs import factory as config_factory
from modeling import factory as model_factory
from hyperparameters import params_dict

with open('configs/yaml/spinenet143_amrcnn.yaml', 'r') as f:
    config = yaml.safe_load(f)


params = config_factory.config_generator('attribute_mask_rcnn')

params = params_dict.override_params_dict(
    params, config, is_strict=True)
# params = params_dict.override_params_dict(
#   params, FLAGS.params_override, is_strict=True)
params.override({
  'architecture': {
      'use_bfloat16': False,  # The inference runs on CPU/GPU.
  },
}, is_strict=True)
params.validate()

In [29]:
# model = AttributeMaskrcnnModel(params)
with tf.Graph().as_default():
    model = model_factory.model_generator(params)
    tfconfig = tf.compat.v1.ConfigProto()
    tfconfig.gpu_options.allow_growth=True
    tfconfig.gpu_options.per_process_gpu_memory_fraction = 0.3
    saver = tf.compat.v1.train.Saver()
    with tf.compat.v1.Session(config=tfconfig) as sess:
        saver.restore(sess, 'checkpoints/fashionpedia-spinenet-143/model.ckpt')
    checkpoint = tf.train.Checkpoint(model=model)
    checkpoint_path = 'checkpoints/fashionpedia-spinenet-143/model.ckpt'
    checkpoint.restore(checkpoint_path).expect_partial()  # Restore TF1.x weights into TF2.x model


ValueError: No variables to save

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import csv
import io

from absl import flags
from absl import logging

import numpy as np
from PIL import Image
from pycocotools import mask as mask_api
import tensorflow.compat.v1 as tf

from dataloader import mode_keys
from configs import factory as config_factory
from modeling import factory as model_factory
from utils import box_utils
from utils import input_utils
from utils import mask_utils
from utils.object_detection import visualization_utils
from hyperparameters import params_dict


  # Load the label map.
label_map_dict = {}
with tf.gfile.Open('datasets/fashionpedia_label_map.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=':')
    for row in reader:
        if len(row) != 2:
          raise ValueError('Each row of the csv label map file must be in '
                           '`id:name` format.')
        id_index = int(row[0])
        name = row[1]
        label_map_dict[id_index] = {
            'id': id_index,
            'name': name,
        }


params = config_factory.config_generator('attribute_mask_rcnn')
if True:
    params = params_dict.override_params_dict(
        params, 'configs/yaml/spinenet143_amrcnn.yaml', is_strict=True)
    params = params_dict.override_params_dict(
      params, '', is_strict=True)
    params.override({
      'architecture': {
          'use_bfloat16': False,  # The inference runs on CPU/GPU.
      },
    }, is_strict=True)
    params.validate()
    params.lock()

model = model_factory.model_generator(params)

with tf.Graph().as_default():
    image_input = tf.placeholder(shape=(), dtype=tf.string)
    image = tf.io.decode_image(image_input, channels=3)
    image.set_shape([None, None, 3])

    image = input_utils.normalize_image(image)
    # image_size = [1108, 792] #[640,640]
    image_size = [640,640]
    image, image_info = input_utils.resize_and_crop_image(
        image,
        image_size,
        image_size,
        aug_scale_min=1.0,
        aug_scale_max=1.0)
    image.set_shape([image_size[0], image_size[1], 3])

    # batching.
    images = tf.reshape(image, [1, image_size[0], image_size[1], 3])
    images_info = tf.expand_dims(image_info, axis=0)

    # model inference
    outputs = model.build_outputs(
        images, {'image_info': images_info}, mode=mode_keys.PREDICT)

    outputs['detection_boxes'] = (
        outputs['detection_boxes'] / tf.tile(images_info[:, 2:3, :], [1, 1, 2]))

    predictions = outputs

    # Create a saver in order to load the pre-trained checkpoint.
    saver = tf.train.Saver()

    image_with_detections_list = []
      
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    config.gpu_options.per_process_gpu_memory_fraction = 0.3


    # with tf.Session(config=config) as sess:
    sess = tf.compat.v1.Session()

    print(' - Loading the checkpoint...')
    saver.restore(sess, 'checkpoints/fashionpedia-spinenet-143/model.ckpt')
    print(saver)
    

C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\keras\legacy_tf_layers\convolutional.py:536: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\keras\engine\base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\keras\legacy_tf_layers\pooling.py:554: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\keras\legacy_tf_layers\core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\keras\legacy_tf_layers\convolutional.py:1660: UserWarning: `tf.layers.conv2d_transpose` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2DTranspose` instead.
  warnings.warn('`tf.layers.conv2d_transpose` is deprecated and '


 - Loading the checkpoint...
INFO:tensorflow:Restoring parameters from checkpoints/fashionpedia-spinenet-143/model.ckpt


INFO:tensorflow:Restoring parameters from checkpoints/fashionpedia-spinenet-143/model.ckpt


In [3]:
vars(saver)

{'_var_list': [<tf.Variable 'spinenet/conv2d/kernel:0' shape=(7, 7, 3, 64) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization/gamma:0' shape=(64,) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization/beta:0' shape=(64,) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization/moving_mean:0' shape=(64,) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization/moving_variance:0' shape=(64,) dtype=float32>,
  <tf.Variable 'spinenet/conv2d_1/kernel:0' shape=(1, 1, 64, 256) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization_1/gamma:0' shape=(256,) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization_1/beta:0' shape=(256,) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization_1/moving_mean:0' shape=(256,) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization_1/moving_variance:0' shape=(256,) dtype=float32>,
  <tf.Variable 'spinenet/conv2d_2/kernel:0' shape=(1, 1, 64, 64) dtype=float32>,
  <tf.Variable 'spinenet/batch_normalization_2/g